## Imports

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import pyLDAvis
import pyLDAvis.lda_model


## Define Stopwords

In [2]:

# Download German language resources for NLTK
nltk.download('stopwords')
nltk.download('punkt')

# Read DataFrame with event descriptions
df = pd.read_csv('data/2000_events_sample.csv')  # Replace 'your_dataframe.csv' with your actual file path or URL

# Get German stop words from NLTK
german_stop_words = set(stopwords.words('german'))
english_stop_words = set(stopwords.words('english'))
self_defined_stop_words = ['stuttgart', 'de', 'www', 'uhr', '00', '30', '19', '20', '21', '22', '23', '00', '10', '11', '12', '13', '14', '1', 'http', 'com', 'br', 'https']

combined_stop_words = german_stop_words.union(english_stop_words).union(self_defined_stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/juliusdoebelt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/juliusdoebelt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Convert the event descriptions to a matrix of token counts 

In [3]:
# Convert event descriptions to document-term matrix
vectorizer = CountVectorizer(stop_words=list(combined_stop_words), max_features=1000, max_df=0.85)

event_descriptions = []
for index, row in df.iterrows():
    event_description = row['description']
    if isinstance(event_description, str):
        event_descriptions.append(event_description)
    else:
        event_descriptions.append('Anderes')

X = vectorizer.fit_transform(event_descriptions)

## Apply Latent Dirichlet Allocation (LDA)

In [4]:
num_topics = 5  # Adjust based on the desired number of categories
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
document_topics = lda.fit_transform(X)

# Get the most probable topic for each document (event)
predicted_labels = document_topics.argmax(axis=1)

## Visualize the topics

In [5]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.lda_model.prepare(lda, X, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3     -120.953606   82.993996       1        1  26.611824
2       36.355686   98.823593       2        1  25.445640
4      -15.591500 -104.074203       3        1  23.862734
0     -170.772110  -73.614441       4        1  13.097670
1      128.276886  -37.030693       5        1  10.982131, topic_info=         Term        Freq       Total Category  logprob  loglift
523      kurs  130.000000  130.000000  Default  30.0000  30.0000
970      yoga  126.000000  126.000000  Default  29.0000  29.0000
217        dj  136.000000  136.000000  Default  28.0000  28.0000
101      band  177.000000  177.000000  Default  27.0000  27.0000
611  mittwoch  347.000000  347.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
611  mittwoch   42.570898  347.164620   Topic5  -4.8114   0.1103
311    finden   34.072509  109.802469   Topic5  -5.0340   1.0387
389      gibt   35.961694  295.634613   Topic5  -4.9801   0.1022
180     dabei   35.484671  237.449009   Topic5  -4.9934   0.3080
14         17   33.763124  160.228186   Topic5  -5.0432   0.6517

[326 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
5         1  0.037210       05
5         2  0.099226       05
5         3  0.248065       05
5         4  0.570549       05
5         5  0.037210       05
...     ...       ...      ...
993       4  0.042458     zwei
993       5  0.118883     zwei
997       3  0.044526     üben
997       5  0.890525     üben
999       5  0.978295  übungen

[655 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 5, 1, 2])

In [6]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = f"Topic #{topic_idx}: "
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

n_top_words = 20
tf_feature_names = vectorizer.get_feature_names_out()
print_top_words(lda, tf_feature_names, n_top_words)

# top words for sample of 2000 events
# Topic #0: dj ab mittwoch club gibt 50 line facebook artists 2022 eintritt eur tickets party heute 16 euro geht 05 könnt
# Topic #1: kurs yoga ab körper hast bewegung anmeldung salsa 18 bachata bitte mittwoch sowie übungen online asanas meditation eintritt frei möglich
# Topic #2: mittwoch freuen 2019 anmeldung 18 findet baden württemberg gerne statt infos gibt 17 treffen mehr online weitere bitte themen ab
# Topic #3: musik live band tour album mal dabei songs ganz jazz immer mehr seit bühne jahren wurde schon jahr zwei welt
# Topic #4: veranstaltung sowie menschen dr leben eintritt benz jahren mercedes kinder museum neue social innen ab fragen anmeldung mehr media ausstellung

# Map the topic labels to your predefined categories
topic_labels_mapping = {
    0: 'Elektrisierend',
    1: 'Körperbewusst',
    2: 'Gesellig',
    3: 'Künstlerisch',
    4: 'Informativ'
}

# Map the predicted labels to category names
predicted_category_labels = [topic_labels_mapping[label] for label in predicted_labels]

# Add the predicted category labels to the DataFrame
df['stimmung'] = predicted_category_labels

# Print or further analyze the predicted category labels
print(df[['description', 'stimmung']])

df.to_csv('data/2000_events_sample.csv', index=False)

Topic #0: dj ab mittwoch club gibt 50 line facebook artists 2022 eintritt eur tickets party heute 16 euro geht 05 könnt
Topic #1: kurs yoga ab körper hast bewegung anmeldung salsa 18 bachata bitte mittwoch sowie übungen online asanas meditation eintritt frei möglich
Topic #2: mittwoch freuen 2019 anmeldung 18 findet baden württemberg gerne statt infos gibt 17 treffen mehr online weitere bitte themen ab
Topic #3: musik live band tour album mal dabei songs ganz jazz immer mehr seit bühne jahren wurde schon jahr zwei welt
Topic #4: veranstaltung sowie menschen dr leben eintritt benz jahren mercedes kinder museum neue social innen ab fragen anmeldung mehr media ausstellung
                                            description        stimmung
0     Jeden Mittwoch best Mix Salsa, Bachata und Kiz...   Körperbewusst
1     Lesung und Gespräch\nModeration: Helmut Böttig...      Informativ
2     Maulina ist sauer. Gemeinsam mit ihrer Mutter ...    Künstlerisch
3     Best Of B-TIGHT Tour 2020\n\